# Design

The problem entails extracting insights from a large JSON file, such as top dates with the most tweets, most used emojis, and most influential users. The challenge lies in efficiently processing this large dataset, and 2 main concerns arise: optimizing memory usage vs. optimizing execution time.
Taking this into account, we can inspect the problem from these 2 points of view.

## Execution time

Here, the question of CPU-bound vs. I/O bound arises. Let's analyze these aspects in the problem:

- CPU-bound tasks: to find the requested insights, some necessary operations on tweets will be: sort, search by key, compare, parse attributes, regex matching, merging control data structures such as counters, keeping track of them, etc.
- I/O tasks: the major task of this type is reading the file, and although it can be considered not-so-small, the data processing does not require writing back to disk, file merging or splitting, transferring through network or similar.

Hence, we can state that this is a **data processing** problem, for which we choose a **CPU-bound** solution to this: Python's `multiprocessing` module. This module allows a developer to spawn processes using high-level abstractions, and also by using processes we can bypass the GIL as each process has its own Python interpreter (a well-known Python issue). This leaves behind other approaches such as `async.io` or `concurrent.futures`.

Using this module we can divide the file tweets into tweet batches, have each worker process its batch, and merge each result. Of course, some operations can be parallelized (e.g.: creating control structures from a tweet) and some cannot (e.g.: sorting). This is similar to what is done in MapReduce engines.

## Memory usage

The main bottleneck of this problem is handling a 398MBs file without creating leaks, that could come from inefficient handling of the file, such as reading the entire file into memory, storing unnecessary data structures, traversing it too many times, not keeping correct track of data points , etc.

To overcome this, we can take advantage of several Python built-in capacities:

- Streaming file data: generators allow us to read the file in chunks and process each data point on-demand, i.e.: we can process tweets as we need them instead of loading everything into memory.
- Lazy evaluation: `yield` lets us create code that will be evaluated only when we ask for it.
- Efficient data structures: the solution heavily relies on `collections.Counter`, a `dict` subclass where elements are keys and counts are values, similar to a bag where there are no duplicates, but also keeping track of the count. This is memory-efficient because operations on it are O(1) and does not save elements that are absent or duplicate.

### Summary

For the memory optimized functions (`q*_memory()`) the approach is streaming the file into memory, processing each tweet as it comes, create the control structures needed for the calculation and, when the stream is ended, finally release it and perform calculation.
For the time optimized functions (`q*_time()`) the approach is splitting tweets into batches, assign each batch to a worker, perform all parallelizable  operations on them (_map_) and then merge the results and perform everything operation that's been left out (_reduce_).

## Possible improvements

- In-place sorting for smaller memory footprint (e.g.: with effectful `list.sort()`).
- Stream batches in each worker process instead of loading everything into memory and then splitting the work.
- Partitioning or indexing data for more efficient reading.

## Side notes

- Why didn't I use cloud?: for this challenge in particular, I felt that developing raw algorithms and using nothing but as pure Python as I could represented better what the esence is of optimizing for memory vs. execution time. In general, all data libraries and frameworks end up on similar conclusions: for optimizing memory we need **reading data as-needed** and **efficient data structures**, for optimizing time we need **parallelizable operations** and **horizontal scaling**. Nevertheless, the spirit of the solution could be taken to a PySpark job in GCP Dataproc or AWS EMR for time optimization, or to AWS Lambda for memory optimization, where you could set the memory limit thinking about what it takes to process 1 stream of data instead of the whole dataset. This type of data could be easily taken to S3, and for more capacities to a document db or even BigQuery, where you could also use SQL for insights.

- Solutions are not completely isolated, time optimized code can use things that are also used in memory optimized code an viceversa.

- Assumptions:
    - For `q3_*`, users mentioned in quoted or retweeted tweets are not taken into account, it's assumed that all mentions in quotes or retweets eventually appear in the document as their own tweet, for the sake of simplicity.

In [ ]:
%load_ext memory_profiler

from src.q1.q1_memory import q1_memory
from src.q1.q1_time import q1_time
from src.q2.q2_memory import q2_memory
from src.q2.q2_time import q2_time
from src.q3.q3_memory import q3_memory
from src.q3.q3_time import q3_time

file_path = 'farmers-protest-tweets-2021-2-4.json'

# q1

## Output

In [ ]:
q1_time_result = q1_time(file_path)
q1_memory_result = q1_memory(file_path)

print(f'q1_time     : {q1_time_result}')
print(f'q1_memory   : {q1_memory_result}')

## Execution time

In [ ]:
%prun -s cumulative -l 10 q1_memory(file_path)

In [ ]:
%prun -s cumulative -l 10 q1_time(file_path)

## Memory usage

In [ ]:
%mprun -f q1_time q1_time(file_path)
%memit q1_time(file_path)

In [ ]:
%mprun -f q1_memory q1_memory(file_path)
%memit q1_memory(file_path)

# q2

## Output

In [ ]:
q2_time_result = q2_time(file_path)
q2_memory_result = q2_memory(file_path)

print(f'q2_time     : {q2_time_result}')
print(f'q2_memory   : {q2_memory_result}')

## Execution time

In [ ]:
%prun -s cumulative -l 10 q2_memory(file_path)

In [ ]:
%prun -s cumulative -l 10 q2_time(file_path)

## Memory usage

In [ ]:
%mprun -f q2_time q2_time(file_path)
%memit q2_time(file_path)

In [ ]:
%mprun -f q2_memory q2_memory(file_path)
%memit q2_memory(file_path)

# q3

## Output

In [ ]:
q3_time_result = q3_time(file_path)
q3_memory_result = q3_memory(file_path)

print(f'q3_time     : {q3_time_result}')
print(f'q3_memory   : {q3_memory_result}')

## Execution time

In [ ]:
%prun -s cumulative -l 10 q3_memory(file_path)

In [ ]:
%prun -s cumulative -l 10 q3_time(file_path)

## Memory usage

In [ ]:
%mprun -f q3_time q3_time(file_path)
%memit q3_time(file_path)

In [ ]:
%mprun -f q3_memory q3_memory(file_path)
%memit q3_memory(file_path)